In [4]:
import numpy as np
import pandas as pd

In [5]:
mapping_base_path = './data/'
city_path = mapping_base_path + 'city.en.txt'
region_path = mapping_base_path + 'region.en.txt'
tag_path = mapping_base_path + 'user.profile.tags.en.txt'
city_mapping = {str(x[0]):x[1] for x in np.array(pd.read_csv(city_path, header=None, sep='\t'))}
region_mapping = {str(x[0]):x[1] for x in np.array(pd.read_csv(region_path, header=None, sep='\t'))}
tag_mapping = {str(x[0]):x[1] for x in np.array(pd.read_csv(tag_path, header=None, sep='\t'))}

In [6]:
header = ['click','weekday','hour','timestamp','log_type',
          'user_id','user_agent','ip','region','city','ad_exchange',
          'domain','url','anonymous_url_id','ad_slot_id','ad_slot_width',
          'ad_slot_height','ad_slot_visibility','ad_slot_format','ad_slot_floor_price',
          'creative_id','key_page_url','adversiter_id','user_tags']
raw_data = pd.read_csv('./data/data_train.txt', header=None, sep='\t')

In [7]:
field_cates = {x:set() for x in header if x not in set(['anonymous_url_id'])}

In [8]:
raw_data.columns = header

In [9]:
Y = np.array(raw_data['click'])
X = np.array(raw_data[header[1:len(header)]])
X_header = header[1:len(header)]
print X.shape
print Y.shape

(2847802, 23)
(2847802,)


In [10]:
# region_city_pair = []
# os_browser_pair = []
# for x in X:
#     region_index = x[7]
#     city_index = x[8]
#     if city_mapping.get(str(city_index)) is not None and  region_mapping.get(str(region_index)) is not None:
#         city = 'city_' + city_mapping.get(str(city_index))
#         region = 'region_' + region_mapping.get(str(region_index))
#         region_city_pair.append((region,city))
#     user_agent = x[5]
#     value = user_agent.split('_')
#     os_browser_pair.append(('user_agent_os_'+value[0],'user_agent_browser_'+value[1]))
# region_city_pair = list(set(region_city_pair))
# f = open('./data/questions.txt','w')
# f.write(":region-city\n")
# for i in range(len(region_city_pair)):
#     for j in range(i+1,len(region_city_pair)):
#         content = ' '.join([region_city_pair[i][0],region_city_pair[i][1],region_city_pair[j][0],region_city_pair[j][1]])
#         content += '\n'
#         f.write(content)
# f.close()

In [18]:
def clean_by_line(line, header):
    rst = []
    non_use_keys = set(['anonymous_url_id'])
    for key,value in zip(header,line):
        value = str(value)
        key = str(key)
        field = field_cates.get(key)
        if value == '' or key in non_use_keys:
            continue
        elif key == 'city':
            if city_mapping.get(str(value)) is not None:
                value = city_mapping.get(str(value))
                rst += [key+'_'+value]
                field.add(key+'_'+value)
        elif key == 'region':
            if region_mapping.get(str(value)) is not None:
                value = region_mapping.get(str(value))
                rst += [key+'_'+value]
                field.add(key+'_'+value)
        elif key == 'timestamp':
            rst += ['time_day_'+value[6:8]]
            field.add(key+'_'+value)
        elif key == 'user_agent':
            value = value.split('_')
            rst += ['user_agent_os_'+value[0]]
            rst += ['user_agent_browser_'+value[1]]
            field.add('user_agent_os_'+value[0])
            field.add('user_agent_browser_'+value[1])
        elif key == 'user_tags':
            for sub_value in value.split(','):
                rst += ['user_tags_'+str(tag_mapping.get(str(sub_value)))]
                field.add('user_tags_'+str(tag_mapping.get(str(sub_value))))
        else:
            rst += [key+'_'+value]
            field.add(key+'_'+value)             
    return rst

In [19]:
X_1 = []
for x in X:
    X_1.append(clean_by_line(x,X_header))

In [20]:
# np.random.shuffle(np.array(X_1))
for key in field_cates.keys():
    print(key,len(field_cates.get(key)))

('domain', 15146)
('ad_slot_width', 11)
('ip', 489690)
('adversiter_id', 1)
('key_page_url', 2)
('click', 0)
('city', 334)
('user_id', 2641292)
('ad_slot_format', 3)
('user_tags', 44)
('creative_id', 11)
('user_agent', 15)
('timestamp', 2434838)
('ad_slot_visibility', 4)
('log_type', 1)
('ad_slot_height', 6)
('hour', 24)
('url', 715316)
('region', 35)
('ad_exchange', 3)
('weekday', 7)
('ad_slot_floor_price', 197)
('ad_slot_id', 51529)


In [22]:
# # np.array(X_1)
# for i in range(10):
#     print len(X_1[i])
# X_1[0:10]
X = X_1

In [23]:
# X[0:10]

In [24]:
import itertools
from collections import Counter
X_index = np.copy(X)
print len(X_index)

2847802


In [25]:
counts = Counter(itertools.chain(*X_index))

In [26]:
MIN_THRESHOLD = 10

In [27]:
# Mapping from index to word
filtered_feature_list = [x[0] for x in counts.most_common() if x[1] > MIN_THRESHOLD and x[1] < len(X_index)]
filtered_feature_set = set(filtered_feature_list)
# Mapping from word to index
inverse_index = {x: i for i, x in enumerate(filtered_feature_list)}

In [28]:
print(len(inverse_index.keys()))

72746


In [33]:
# inverse_index

In [35]:
# field_cates['']
field_cates_index = {}
for key in field_cates.keys():
    field_cates_index[key] = [inverse_index.get(f) for f in field_cates.get(key) if inverse_index.get(f) is not None]
#     print(key,len(field_cates.get(key)))

In [37]:
# field_cates_index

In [38]:
for key in field_cates.keys():
    print(key,len(field_cates.get(key)))
print('-----------------')
for key in field_cates_index.keys():
    print(key,len(field_cates_index.get(key)))

('domain', 15146)
('ad_slot_width', 11)
('ip', 489690)
('adversiter_id', 1)
('key_page_url', 2)
('click', 0)
('city', 334)
('user_id', 2641292)
('ad_slot_format', 3)
('user_tags', 44)
('creative_id', 11)
('user_agent', 15)
('timestamp', 2434838)
('ad_slot_visibility', 4)
('log_type', 1)
('ad_slot_height', 6)
('hour', 24)
('url', 715316)
('region', 35)
('ad_exchange', 3)
('weekday', 7)
('ad_slot_floor_price', 197)
('ad_slot_id', 51529)
-----------------
('domain', 3814)
('ad_slot_format', 3)
('ip', 43222)
('adversiter_id', 0)
('key_page_url', 2)
('click', 0)
('city', 334)
('user_id', 14)
('ad_slot_width', 11)
('user_tags', 44)
('creative_id', 11)
('user_agent', 15)
('timestamp', 0)
('ad_slot_visibility', 4)
('log_type', 0)
('ad_slot_height', 6)
('hour', 24)
('url', 18386)
('region', 35)
('ad_exchange', 3)
('weekday', 7)
('ad_slot_floor_price', 97)
('ad_slot_id', 6707)


In [46]:
field_cates_index_f = open('./data/ipinyou/field_cates_index_not_aligned.csv','w')
for key in field_cates_index.keys():
    if(len(field_cates_index.get(key)) > 0):
        field_cates_index_f.write(key+'\t'+','.join([str(i) for i in field_cates_index.get(key)])+'\n')

In [43]:
f_reverse_dict = open('./data/ipinyou/reverse_dictionary_not_aligned.csv','w')
for key in inverse_index.keys():
    f_reverse_dict.write(key+','+str(inverse_index.get(key))+'\n')
f_reverse_dict.close()

In [44]:
print len(Y),len(X)

2847802 2847802


In [47]:
f = open('./data/ipinyou/training_not_aligned.csv','w')
f_label = open('./data/ipinyou/labels.csv','w')
count = 0
X = np.array(X)
for i in range(len(X)):
    line = str(','.join([str(inverse_index.get(x)) for x in X[i] if inverse_index.get(x) != None]))
    f.write(line+'\n')
    f_label.write(str(Y[i]) + '\n')
    count += 1
f.close()
print count

2847802
